In [1]:
from dash import Dash, html, dcc, Input, Output, callback, dash_table
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
import seaborn as sns
from scipy import stats
plt.rc('font', family='Malgun Gothic')

In [29]:
# 데이터 불러오기

df = pd.read_excel('data/교육통계데이터_2015_2022_최종.xlsx') 

In [4]:
# 학업중단자와 교사수 비율의 소수점이 너무 많아 데이터를 확인하기 힘들어 소수점 둘째짜리까지로 수정함 

In [5]:
df['학업중단자_비율'] = round(df['학업중단자_비율'], 2)

In [6]:
df['교사수_비율'] = round(df['교사수_비율'], 2)

In [7]:
# 데이터프레임 확인 

df

,연도,시도,행정구역,학제,학교수,학생수_계,교원수_정규_계,교원수_정규_상담_계,학업중단자_계,학업중단자_비율,교사수_비율
0,2015,서울,종로구,고등학교,15,12653,840,8,179,1.41,0.95
1,2015,서울,중구,고등학교,12,8933,637,5,97,1.09,0.78
2,2015,서울,용산구,고등학교,10,7580,538,6,100,1.32,1.12
3,2015,서울,성동구,고등학교,7,6428,467,3,118,1.84,0.64
4,2015,서울,광진구,고등학교,9,10936,586,2,168,1.54,0.34
...,...,...,...,...,...,...,...,...,...,...,...
1471,2020,경남,합천군,고등학교,6,713,109,1,22,3.09,0.92
1472,2020,경남,소계,고등학교,190,89026,7296,71,1301,1.46,0.97
1473,2020,제주,제주시,고등학교,20,14495,976,12,171,1.18,1.23
1474,2020,제주,서귀포시,고등학교,10,4184,383,5,66,1.58,1.31


In [8]:
# 시와 연도별로 학업중단자와 교사수의 비율 평균을 계산함 

In [9]:
df2 = df.groupby(['시도', '연도'])['학업중단자_비율'].mean()
df2

시도  연도  
강원  2015    1.405789
    2016    1.456316
    2017    1.458947
    2018    1.657895
    2019    1.772105
              ...   
충북  2016    1.814167
    2017    2.088333
    2018    2.123333
    2019    2.545833
    2020    2.204167
Name: 학업중단자_비율, Length: 102, dtype: float64

In [10]:
df3 = df.groupby(['시도', '연도'])['교사수_비율'].mean()
df3

시도  연도  
강원  2015    0.372105
    2016    0.433684
    2017    0.482632
    2018    0.858421
    2019    1.016842
              ...   
충북  2016    0.971667
    2017    0.957500
    2018    1.096667
    2019    1.173333
    2020    1.270000
Name: 교사수_비율, Length: 102, dtype: float64

In [11]:
# 두 데이터프레임을 옆으로 합침

In [12]:
df4 = pd.concat([df2, df3], axis=1)

In [13]:
df4 = df4.reset_index() # 인덱스를 재설정하여 시도와 연도를 인덱스에서 빼냄

In [14]:
df4

,시도,연도,학업중단자_비율,교사수_비율
0,강원,2015,1.405789,0.372105
1,강원,2016,1.456316,0.433684
2,강원,2017,1.458947,0.482632
3,강원,2018,1.657895,0.858421
4,강원,2019,1.772105,1.016842
...,...,...,...,...
97,충북,2016,1.814167,0.971667
98,충북,2017,2.088333,0.957500
99,충북,2018,2.123333,1.096667
100,충북,2019,2.545833,1.173333


In [15]:
# 위의 데이터프레임을 복사하여 각각 기준란에 학업중단자의 비율인지 교사수의 비율인지를 표시함 

In [16]:
df5 = df4.copy()

In [17]:
for i in range(len(df4)):
    df4.loc[i,'비율값'] = df4.loc[i, '학업중단자_비율']
    df4.loc[i,'기준'] = "학업중단자"

In [18]:
for i in range(len(df5)):
    df5.loc[i,'비율값'] = df5.loc[i, '교사수_비율']
    df5.loc[i,'기준'] = "교사수"

In [19]:
df4

,시도,연도,학업중단자_비율,교사수_비율,비율값,기준
0,강원,2015,1.405789,0.372105,1.405789,학업중단자
1,강원,2016,1.456316,0.433684,1.456316,학업중단자
2,강원,2017,1.458947,0.482632,1.458947,학업중단자
3,강원,2018,1.657895,0.858421,1.657895,학업중단자
4,강원,2019,1.772105,1.016842,1.772105,학업중단자
...,...,...,...,...,...,...
97,충북,2016,1.814167,0.971667,1.814167,학업중단자
98,충북,2017,2.088333,0.957500,2.088333,학업중단자
99,충북,2018,2.123333,1.096667,2.123333,학업중단자
100,충북,2019,2.545833,1.173333,2.545833,학업중단자


In [20]:
df5

,시도,연도,학업중단자_비율,교사수_비율,비율값,기준
0,강원,2015,1.405789,0.372105,0.372105,교사수
1,강원,2016,1.456316,0.433684,0.433684,교사수
2,강원,2017,1.458947,0.482632,0.482632,교사수
3,강원,2018,1.657895,0.858421,0.858421,교사수
4,강원,2019,1.772105,1.016842,1.016842,교사수
...,...,...,...,...,...,...
97,충북,2016,1.814167,0.971667,0.971667,교사수
98,충북,2017,2.088333,0.957500,0.957500,교사수
99,충북,2018,2.123333,1.096667,1.096667,교사수
100,충북,2019,2.545833,1.173333,1.173333,교사수


In [21]:
# 학업중단자 비율과 교사수 비율 column을 없애고 비율값과 기준만 남겨둠 

In [22]:
df4 = df4.drop(['학업중단자_비율', '교사수_비율'], axis=1)

In [23]:
df5 = df5.drop(['학업중단자_비율', '교사수_비율'], axis=1)

In [24]:
df_graph = pd.concat([df4, df5])
df_graph

,시도,연도,비율값,기준
0,강원,2015,1.405789,학업중단자
1,강원,2016,1.456316,학업중단자
2,강원,2017,1.458947,학업중단자
3,강원,2018,1.657895,학업중단자
4,강원,2019,1.772105,학업중단자
...,...,...,...,...
97,충북,2016,0.971667,교사수
98,충북,2017,0.957500,교사수
99,충북,2018,1.096667,교사수
100,충북,2019,1.173333,교사수


In [25]:
# 대시보드에서 보여줄 테이블을 정리함 
# 테이블은 연도, 시도, 행정구역, 학업중단자 비율과 교사수 비율만 보여줌 

In [26]:
df_table = df[['연도', '시도', '행정구역', '학업중단자_비율', '교사수_비율']]

In [27]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.Div(className='row', children='전문상담교사수와 학업중단자',
             style={'textAlign': 'center', 'color': 'blue', 'fontSize': 30}),

    html.Div(className='row', children=[
        dcc.Dropdown(df_graph['시도'].unique(), '서울', id='dropdown-selection'), 
        # 그래프는 시도를 기준으로 Dropdown하여 보여줌
        # 서울을 먼저 보여줌
        dcc.Graph(figure={}, id='graph-content')     
    ]),dash_table.DataTable(data=df_table.to_dict('records'), page_size=10) 
    # 테이블은 업데이트하지 않고 한번에 전체 시도의 비율을 출력
])

@callback(
    Output(component_id='graph-content', component_property='figure'),
    Input(component_id='dropdown-selection', component_property='value')
)

def update_graph1(value):
    dff = df_graph[df_graph['시도']==value]
    return px.line(dff, x="연도", y="비율값",color='기준', symbol='기준') # 학업중단자와 교사수의 색을 다르게 하여 한번에 출력

if __name__ == '__main__':
    app.run(debug=True)